Imports


In [ ]:
import numpy as np
import pandas as pd
import gym
from gym import Env
from gym.spaces import Box, Discrete, MultiDiscrete
import os
import tensorflow
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import random
from tensorflow.keras.layers import Input
import matplotlib.pyplot as plt
import pickle
from scipy.optimize import minimize
import itertools
import scipy.stats as st
import numpy as np
import mplfinance as mpf

Generating data

In [ ]:
def generate_ohlc_data(means, correlation_matrix, num_days=5040, num_timesteps=24):
    column_names = [f'Stock_{i+1}_{col}' for i in range(len(means)) for col in ['Open', 'High', 'Low', 'Close']]
    ohlc_data = []
    
    # Fill the first row with ones
    ohlc_data.append([1] * len(column_names))

    for day in range(num_days):
        # Generate data for one day
        a = st.multivariate_normal.rvs(means, correlation_matrix, size=(1, num_timesteps))
        a = np.exp(a)
        lity = []
        #print(a)
        
        for timestep in range(len(a[0])):
            close_prev = ohlc_data[-1][3 + timestep * 4]
            #print(np.cumsum(a[:,timestep]) , close_prev)
            
            lity.append(close_prev)
            lity.append(close_prev * max(np.cumprod(a[:, timestep])))  # High
            lity.append(close_prev * min(np.cumprod(a[:, timestep])))  # Low
            lity.append(close_prev * np.cumprod(a[:, timestep])[-1])   # Close
        #print(lity)
        
        ohlc_data.append((lity))
    
    return pd.DataFrame(ohlc_data, columns=column_names)

means = [0.00003, 0.00002, 0.000035]
correlation_matrix = np.array([[0.000034, 0.0000075, -0.00001],
                               [0.0000075, 0.000012, -0.000008],
                               [-0.00001, -0.000008, 0.000045]
])


OHLC = generate_ohlc_data(means=means, correlation_matrix=correlation_matrix)
OHLC = OHLC.drop(OHLC.index[0]).reset_index(drop=True)

OHLC.head()

Standardize Data

In [ ]:
def StandardizeData(data, n):
    if isinstance(data, pd.DataFrame):
        df = data.copy()  # Make a copy to avoid modifying the original DataFrame
    elif isinstance(data, list):
        df = pd.DataFrame(data)  # Convert list to DataFrame
    else:
        raise ValueError("Input data must be a DataFrame or a list.")
    
    Standardized = pd.DataFrame()
    for col in df.columns:
        ST = pd.DataFrame()
        for i in range(n+1):
            stock_num = int(col.split('_')[1])  # Extract the stock number from the column name
            close_col = f'Stock_{stock_num}_Close'  # Get the corresponding 'Close' column
            ST[f'{col}_{i}'] = df[col].shift(-i) / df[close_col].shift(-n)  # Divide by the corresponding 'Close' column
        Standardized = pd.concat([Standardized, ST], axis=1)
    
    # Drop rows with NaN values
    


    Ratio = pd.DataFrame()
    close_columns = [col for col in df.columns if 'Close' in col]
    for col in close_columns:
        
        Ratio[col] = np.log(df[col].shift(-n-1) /df[col].shift(-n) )
        Ratio.dropna()


    return Standardized[:-n-1], Ratio[:-n-1]

InputStates, RewardVector = StandardizeData(OHLC, 4)
print(RewardVector.iloc[:2529].cov() , RewardVector.iloc[:2529].mean())

Tangency Portfolio calculations:


In [ ]:

def tangency_weights(covariance_matrix, mean_vector):
    n = len(mean_vector)
    
    # Objective function to maximize (Sharpe ratio)
    def objective_function(weights):
        portfolio_return = np.dot(weights, mean_vector)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_std
    
    # Constraint: sum of weights equals 1
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(np.abs(weights)) - 1})
    
    # Bounds for each weight (-1 <= weight <= 1)
    bounds = tuple((-1, 1) for _ in range(n))
    
    # Initial guess for weights
    initial_guess = np.ones(n) / n
    
    # Optimization
    result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Extracting optimal weights
    optimal_weights = result.x
    
    # Calculate the mean and standard deviation of the tangency portfolio
    tangency_return = np.dot(optimal_weights, mean_vector)
    tangency_std = (np.dot(optimal_weights.T, np.dot(covariance_matrix, optimal_weights)))
    
    return {
        'weights': optimal_weights,
        'mean': tangency_return,
        'std': tangency_std,
        'maximized_value': -result.fun  # Negative because we minimized the negative of Sharpe ratio
    }

# Example usage:
# Assuming RewardVector is a DataFrame containing your data

result = tangency_weights(RewardVector.cov(), RewardVector.mean())
print("Optimal Weights:", result['weights'])
print("Tangency Portfolio Mean:", result['mean'] * 100)  # Assuming mean is annualized
print("Tangency Portfolio Standard Deviation:", result['std'])
print("Maximized Value:", result['maximized_value'])

Replay Buffer and Code

In [ ]:
class ReplayBuffer():
    def __init__(self, max_size , input_shape, n_actions, discrete= True):
        self.mem_size = max_size
        self.input_spahe =input_shape
        self.dis = discrete
        self.mem_counter = 0
        self.state_mem =np.zeros((self.mem_size , input_shape))
        self.new_state_mem = np.zeros((self.mem_size , input_shape))

        dtype = np.int16 if self.dis else np.float32
        self.action_mem = np.zeros((self.mem_size , n_actions), dtype=dtype)
        self.reward_mem = np.zeros((self.mem_size))
        self.terminal_mem = np.zeros(self.mem_size, dtype=np.float32) #he has this as float???

    def Store_transition(self, state, action , reward , next_state , done):
        index = self.mem_counter%self.mem_size
        self.state_mem[index] = state
        self.new_state_mem[index] = next_state
        self.action_mem[index] = action
        self.reward_mem[index] = reward
        self.terminal_mem[index] = 1- int(done)

        if self.dis:
            actions = np.zeros(self.action_mem.shape[1])
            actions[action] =1.0
            self.action_mem[index] = actions
        else:
            self.action_mem[index] = action
        self.mem_counter+=1

    def Sample_buffer(self, batch_size):
        Max_mem = min(self.mem_counter , batch_size)
        batch = np.random.choice(Max_mem, batch_size , replace= False)
        states = self.state_mem[batch]
        states_ = self.new_state_mem[batch]
        action = self.action_mem[batch]
        reward = self.reward_mem[batch]
        EpisodeEnds = self.terminal_mem[batch]

        return states, action, reward,states_, EpisodeEnds
    
class ContextualBandit(object):
    
    def __init__(self, state_dim, action_dim,steps, transform = None, gamma=0.99, alpha=0.0001, epsilon=1, decay=0.99999,
                  decaya=0.9 , cf1 = 64 , cf2 = 64, batch_size = 15000, max_mem = 750000, model_file = 'dqn_model.h5' , tau =0.05 , eta = 1e-2 ):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.batch_size = batch_size
        self.maxmem = max_mem
        self.steps = steps
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        self.model_file = model_file
        self.eps = epsilon
        self.decay = decay
        self.decaya = decaya
        self.actions = self.generate_grid(action_dim-1, self.steps, transform)
        self.action_dim = len(self.actions)
        self.tau = tau
        self.cf1 = cf1
        self.cf2 = cf2
        self.last_vt = 0
        self.last_wt = 0
        self.eta = eta
        self.last_sr = 0
        self.count = 0
        self.sr = 0
        self.etas = 1
        self.model = self.build_model()  # Initialize model after num_tilings
        self.target = self.build_model()
        self.Memory = ReplayBuffer(max_mem, state_dim, self.action_dim , discrete= True)
        self.Update_NN_Parameters(tau=1)
    
    def generate_grid(self, n, m, offset=None):
        # Generate all possible combinations of bin sizes
        points = []
        for combination in itertools.product(*[range(-m, m + 1) for _ in range(n)]):
            if all(abs(x) <= m for x in combination) and all(-1 <= x / m <= 1 for x in combination):
                points.append(combination)

        points = np.array(points) / m
        
        if offset is not None:
            points += np.array(offset)
        
        # Filter out points where the sum of absolute values is greater than 1
        points = [point for point in points if sum(abs(x) for x in point) <= 1]
        
        return np.array(points)
        
    

    def build_model(self):
        model = Sequential([
            LSTM(self.cf1, activation='relu', input_shape=(None, self.state_dim)),
            Dense(self.action_dim, activation='softmax')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.alpha))
        return model

        
    def Remember(self, state, action, reward, next_state, done):
        
        self.Memory.Store_transition(state,action, reward, next_state, done)
    
    def select_action(self, state, bool=True):
        
        if max(self.eps, 0.01) > np.random.random() and bool:
            self.eps *= self.decay
            return np.random.randint(self.action_dim)
        else: 
            
            action_probabilities = self.model.predict(state)
            action_index = np.argmax(action_probabilities)
            return action_index
        
    def Update_NN_Parameters(self, tau=None):
        if tau is None:
            tau = self.tau
            
        actor_weights = self.model.get_weights()
        actor_target_weights = self.target.get_weights()

        updated_actor_target_weights = []

        for actor_weight, target_weight in zip(actor_weights, actor_target_weights):
            updated_weight = actor_weight * tau + target_weight * (1 - tau)
            updated_actor_target_weights.append(updated_weight)

        self.target.set_weights(updated_actor_target_weights)


    def learn(self):
        if self.Memory.mem_counter < self.batch_size:
            return 0, 0, 0  # Returning an additional 0 for TD error
        
        # Sample minibatch from replay buffer
        state, action, reward, new_state, done = self.Memory.Sample_buffer(self.batch_size)
        state = state.reshape(self.batch_size, 1, self.state_dim)
        new_state = new_state.reshape(self.batch_size, 1, self.state_dim)
        
        # Compute Q-values for current and next states
        q_eval = self.model.predict(state)
        q_next = self.model.predict(new_state)
        q_target = self.target.predict(state)
        
        # Compute target Q-values using the target network
        q_next_target = self.target.predict(new_state)
        
        # Compute action indices
        actionind = np.argmax(action, axis=1)
        
        # Define batch indices
        batchindex = np.arange(self.batch_size)
        
        # Update target Q-values less frequently
        if self.count % 450 == 0:
            self.Update_NN_Parameters()
        
        # Calculate TD error
        td_target = reward + self.gamma * np.max(q_next_target, axis=1) * (1 - done)
        td_error = td_target - q_eval[batchindex, actionind]
        
        # Update Q-network
        q_target[batchindex, actionind] = td_target
        
        # Train the main Q-network
        _ = self.model.fit(state, q_target, verbose=0)
        
        # Compute Q-values after update
        q_eval_after_update = self.model.predict(state)
        self.alpha *= self.decaya
        
        return q_eval, q_eval_after_update, td_error
    
    def save_model(self):
        self.model.save(self.model_file)

    def _tiny(self):
        return np.finfo('float64').eps
    
    
    def calculate_dsr(self, rt):
        self.sr = self.last_sr
        if self.count < 1 / self.eta:
            self.etas = 1/(self.count + 1)
            changeA = (-self.last_vt + rt) / (self.count + 1)
            changeB = (-self.last_wt + rt ** 2) / (self.count + 1)
        else:
            changeA =  self.eta * (rt - self.last_vt)
            changeB =  self.eta * (rt ** 2 - self.last_wt)
            self.etas = self.eta
        if self.last_wt - self.last_vt ** 2 != 0:
            self.last_sr = (self.last_wt*changeA/self.etas -0.5*self.last_vt*changeB/self.etas)/((self.last_wt-self.last_vt**2)**(1.5))
        else:
            self.last_sr = 0
        self.count += 1
        self.last_wt += changeB
        self.last_vt += changeA
        return self.last_sr 
    
    def reset(self):
        self.count = 0
        self.last_sr = 0
        self. last_vt = 0
        self.last_wt = 0
        self.etas = 1
    def GetReward(self, action, rewards):
        act = self.actions[action%(len(self.actions))]

        if np.sum(np.abs(act)) >1:
            return - 10000000000000
        else:
            if action < len(self.actions): #the last one is a buy
                acti = np.append(act , 1-np.sum(np.abs(act)))
            else: # For a sell of the last.
                acti = np.append(act , np.sum(np.abs(act))-1)
        
        reward = np.array(rewards)
        Rt = np.dot(reward,np.array(acti))
        reward = self.calculate_dsr(Rt)
        
        return reward 

Training Loop

In [ ]:
# Concatenate the two DataFrames along axis 1 (columns)
df = InputStates

# Optional: Reset the index of the concatenated DataFrame
df.reset_index(drop=True, inplace=True)
state_dim = df.shape[1] + 3
action_dim = RewardVector.shape[1]
agent = ContextualBandit(state_dim, action_dim, alpha=0.00000001,steps=20, decay=0.999, batch_size=15000, eta=0.01)

In [ ]:
# Initialize lists to store evaluation results
eval = []
eval2 = []
# Concatenate the two DataFrames along axis 1 (columns)
df = InputStates

# Optional: Reset the index of the concatenated DataFrame
df.reset_index(drop=True, inplace=True)

state_dim = df.shape[1] + 3
action_dim = RewardVector.shape[1]
agent = ContextualBandit(state_dim, action_dim, alpha=0.000000001,steps=20, decay=0.999, batch_size=15000, eta=0.01)
globalaction = 500 #example golbal action



model_path = "RealData3stocksV1.pkl"

save_interval = 500  # Save the model every 1000 iterations

for j in range(50):
    a = np.random.randint(110 , len(df)- 2000)
    
    agent.reset()
    for i in range(100):
        action = globalaction
        agent.GetReward(action, RewardVector.iloc[i + a])
    for i in range(500):
        state = df.iloc[i + a]*100
        state_array = state.values.reshape(1, -1)
        result = np.hstack((state_array, np.array([[agent.last_vt, agent.last_wt, agent.etas]]).reshape(1, -1)))[0]
        result = result.reshape(1, 1, df.shape[1]+3)
        
        action = agent.select_action(result, bool=True)
        reward = agent.GetReward(action, RewardVector.iloc[i + a])
        

        next_state = df.iloc[i + a + 1]*100
        next_state_array = next_state.values.reshape(1, -1)
        result2 = np.hstack((next_state_array, np.array([[agent.last_vt, agent.last_wt, agent.etas]]).reshape(1, -1)))[0]
        result2 = result2.reshape(1, 1, df.shape[1]+3)
        done = False

        if i % 50 == 0 or agent.count == 500:
            agent.Remember(result, action, reward, result2, done)
            e1, e2 , td = agent.learn()

            if isinstance(e1, np.ndarray) and isinstance(e2, np.ndarray):
                eval.append(np.mean(abs(td)))
                eval2.append(np.mean(e2))
            

        agent.Remember(result, action, reward, result2, done)

        if i % save_interval == 0:
            # Save the model periodically
            with open(model_path, 'wb') as f:
                pickle.dump(agent, f)
            print(f"Model saved at iteration {i} as {model_path}")

plt.plot(eval)
plt.xlabel('Iterations')
plt.ylabel('Rewards')
plt.title('Reward Trend over Training')
plt.show()

Verification Loop

In [ ]:
test_actions = []
test_rewards = []
rew = []
df = InputStates
# Testing loop
agent.reset()
a = 0
for i in range(len(df) - 600, len(df)-500):
    action = 178
    agent.GetReward(action, RewardVector.iloc[i + a])

for i in range(len(df) - 500, len(df)):
    state = df.iloc[i + a]
    state_array = state.values.reshape(1, -1)*100
    result = np.hstack((state_array, np.array([[agent.last_vt, agent.last_wt, agent.etas]]).reshape(1, -1)))[0]
    result = result.reshape(1, 1, df.shape[1] + 3)
    action = agent.select_action(result, bool=True)
    test_actions.append(action)
    reward = agent.GetReward(action, RewardVector.iloc[i + a])
    test_rewards.append(reward)
    actie = agent.actions[action%len(agent.actions)]
    if action < len(agent.actions): #the last one is a buy
        acti = np.append(actie , 1-np.sum(np.abs(actie)))
    else: # For a sell of the last.
        acti = np.append(actie , np.sum(np.abs(actie))-1)
        
    Rt = np.dot(np.array(RewardVector.iloc[i + a]),np.array(acti))
    #print(np.array(RewardVector.iloc[i + a]),np.array(acti))
    rew.append(Rt)
        

# Convert test_rewards to a numpy array for calculations
rew = np.array(rew)

# Calculate Sharpe Ratio
# Calculate average return
average_return = np.mean(rew)

risk_free_rate = 0

# Calculate standard deviation of returns
std_dev_returns = np.std(rew)

# Compute the Sharpe Ratio
sharpe_ratio = (average_return - risk_free_rate) / std_dev_returns

# Convert log-normal returns to simple returns for cumulative calculations
simple_returns = np.exp(rew) - 1

# Compute cumulative returns
cumulative_returns = np.cumprod(1 + simple_returns)

# Calculate drawdowns
peak = np.maximum.accumulate(cumulative_returns)
drawdowns = (( cumulative_returns- peak) / peak)

# Find the maximum drawdown
max_drawdown = np.min(drawdowns)

# Print results
print("Actions taken during testing:", test_actions)
print("Rewards obtained during testing:", test_rewards)
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Maximum Drawdown: {max_drawdown:.4f}")

# Plot rewards
plt.plot(test_rewards)
plt.title("Rewards Obtained During Testing")
plt.xlabel("Time")
plt.ylabel("Reward")
plt.show()

# Plot cumulative returns
plt.plot(cumulative_returns)
plt.title("Cumulative Returns")
plt.xlabel("Time")
plt.ylabel("Cumulative Return")
plt.show()

# Plot drawdowns
plt.plot(drawdowns)
plt.title("Drawdowns")
plt.xlabel("Time")
plt.ylabel("Drawdown")
plt.show()

from collections import Counter
action_counts = Counter(test_actions)


# Determine the five most common actions
most_common_actions = action_counts.most_common(5)

# Calculate the percentage of time each of these actions appears
total_actions = len(test_actions)
most_common_actions_percentages = [(action, count, (count / total_actions) * 100) for action, count in most_common_actions]

print(most_common_actions_percentages)

a1 =cumulative_returns
a1draw = drawdowns